In [1]:
import numpy as np 
from sympy import *

## Introducción
Queremos modelar una batalla, y queremos hacerlo de manera muy simple. El modelo que vamos a considerar fué propuesto por Frederick Lanchester en 1916.

### Las suposiciones son las siguientes:

Hay dos lados en la batalla: azules y rojos.
Los principales factores que deciden el resultado de la batalla son
- Hay dos lados en la batalla: **azules** y **rojos**.
- Los principales factores que deciden el resultado de la batalla son el número de tropas y el entrenamiento/equipo.
- Sea $x$ el número de tropas de los **rojos** y $y$ el número de tropas de los **azules**.
- Sea $a$ la potencia de fuego de los **rojos** y sea $b$ la potencia de fuego de los **azules**.
    - La _potencia de fuego_ está basada en el entrenamiento, equipo, etc.
    
Con estas suposiciones tenemos el modelo de Lanchester:

$$
\frac{dx}{dt} = -ay
$$


$$
\frac{dy}{dt} = -bx
$$

con condiciones iniciales $x(0) = x_0$ y $y(0) = y_0$.

- Usando **Simpy** resuelva las ecuaciones de manera explícita.


In [2]:
t = Symbol('t')                          #tiempo
x = Function('x')                        #Número de tropas rojas
dx = Derivative(x(t), t)                 #Cambio de la fuerza (número) de la tropa roja

y1 = Function('y')                        #Número de tropas azules
dy = Derivative(y1(t), t)                 #Cambio de la fuerza (número) de la tropa azul


a = Symbol('a')                          #Potencia de fuego de x
b = Symbol('b')                          #Potencia de fuego de y

x0 = Symbol('x_0')                       #Simbolo de la condición inicial en x
y0 = Symbol('y_0')                       #Símbolo de la condición inicial en y

In [3]:
ode1 = Eq(dx,-a*y1(t))                     #Vamos a declarar las ecuaciones diferenciales para ver si se puede resolver con funciones y/o métodos de Python
ode1

Eq(Derivative(x(t), t), -a*y(t))

In [4]:
ode2 = Eq(dy,-b*x(t))                     #Vamos a declarar las ecuaciones diferenciales para ver si se puede resolver con funciones y/o métodos de Python
ode2

Eq(Derivative(y(t), t), -b*x(t))

In [5]:
dsolve(ode1)  

Eq(x(t), C1 - a*Integral(y(t), t))

In [6]:
dsolve(ode2)  

Eq(y(t), C1 - b*Integral(x(t), t))

In [7]:
#Definimos la función que caracteriza al sistema para poder utilizar otros métodos de resolución

def Lanchester(estado, tiempo,a=a,b=b):
    l1 = -a*estado[1]                          # Si tomamos como condiciones iniciales coordenadas (x,y)
    l2 = -b*estado[0]
    return np.array([l1,l2])

In [8]:
# Usaremos el método de Runge-Kutta 2. Las variables son:
# y -> el estado inicial 
# t -> tiempo
# dt -> cambio en el tiempo
# derivadas -> la función que tiene el sistema

def RK2(y, t, dt, derivadas):
    k0 = dt*derivadas(y, t)
    k1 = dt*derivadas(y + k0, t + dt)
    y_next = y + 0.5*(k0 + k1)
    
    return y_next

In [9]:
N = 100 # -> definimos el número de steps

time = np.linspace(0,100,N)   # -> el tiempo

dt = 100/float(N-1)  # -> La variación del tiempo

y = np.zeros([N,2], dtype=Symbol)   #Este arreglo va a tener las condiciones iniciales y va a guardar las soluciones
y[0,0]=x0
y[0,1]=y0

In [10]:
#Esto hace la resolución del método de Runge-Kutta2

for i in range(0,N-1):
    y[i+1] = RK2(y[i], time[i], dt, Lanchester)

In [11]:
#Aquí vemos que el arreglo si tiene soluciones

y[2,0]

-0.505050505050505*a*y_0 - 0.505050505050505*a*(-1.01010101010101*b*x_0 + y_0) - 0.505050505050505*a*(-0.505050505050505*b*x_0 - 0.505050505050505*b*(-1.01010101010101*a*y_0 + x_0) + y_0) - 0.505050505050505*a*(-0.505050505050505*b*x_0 - 0.505050505050505*b*(-1.01010101010101*a*y_0 + x_0) - 1.01010101010101*b*(-0.505050505050505*a*y_0 - 0.505050505050505*a*(-1.01010101010101*b*x_0 + y_0) + x_0) + y_0) + x_0

- Deduzca e interprete la **Ley de cuadrados de Lanchester**. 
$$
x^2 - \frac{b}{a}y^2 = K,
$$
para varios valores de $K$. ¿Qué sucede cuando $K=0$?

In [12]:
ode1

Eq(Derivative(x(t), t), -a*y(t))

In [13]:
ode2

Eq(Derivative(y(t), t), -b*x(t))

Primero, multiplicamos la primer ecuación por bx y la segunda por ay
$$
(1) \ bx\frac{dx}{dt} = -bxay
$$

$$
(2) \  ay\frac{dy}{dt} =-bxay
$$
Igualamos (1) con (2)
$$
ay\frac{dy}{dt} = bx\frac{dx}{dt}
$$ 
Despejamos

$$
\frac{dy}{dx} = \frac{bx}{ay}
$$ Integramos desde la condición incial a lo largo de un periodo
$$ 
a\int_{y_0}^{y_t} y \mathrm{d}y = b\int_{x_0}^{x^t} x \mathrm{d}x
$$
Por lo que$$
ay_t^{2} - ay_0^{2}= bx_t^{2} - bx_0^{2}
$$
Así$$
ay_t^{2} - bx_t^{2} = ay_0^2 - bx_0^2
\\ $$
$$
 \quad\mathrm{Como}\quad  \frac{ay_0^2 - bx_0^2}{a} \quad\mathrm{constante, tenemos}\quad 
$$$$
(3) \ \  \  \ y_t^{2} - \frac{b}{a}x_t^{2}  = K
$$

- Modele combate entre guerrillas (_GUERCOM_): El territorio es grande y hay muchos lugares dónde esconderse. Las fuerzas **azules** deben de encontrar a las fuerzas **rojas** primero antes de inflingir daños, entre más **rojos** más fácil encontrarlos. 
$$
\frac{dx}{dt} = -axy
$$
$$
\frac{dy}{dt} = -bxy
$$
Explique estas ecuaciones e indique los posibles casos. ¿Qué pasa si $x_0 = 3y_0$?¿Qué tan efectivo deben de ser los **azules** para lograr un empate?

Este modelo representa el combate de 2 fuerzas que no tienen la ubicación de la fuerza contraria. Así mismo, no conocen el daño que han infligido al equipo contrario. Esto ahrá que concentren su fuego en una área determinada en la que creen que se encuentra el enemigo. 

Siendo 'ax' la tasa a la cuál Y aniquila a X. Esta se multiplica por Y, siendo Y la cantidad de elemenots de Y que disparan a X. 
De la misma manera, 'by' la tasa a la cuál X aniquila a Y. Esta se multiplica por X (cantidad de X que dispara a Y)



In [14]:
#VALORES PARA EL MODELO GUERRACOM
tC = Symbol('t')                          #tiempo
xC = Function('x')                        #Número de tropas rojas 
dxC = Derivative(x(t), t)                 #Cambio de la fuerza (número) de la tropa roja

y1C = Function('y')                        #Número de tropas azules
dyC = Derivative(y1(t), t)                 #Cambio de la fuerza (número) de la tropa azul


aC = Symbol('a')                          #Potencia de fuego de x
bC = Symbol('b')                          #Potencia de fuego de y

x0C = Symbol('x_0')                       #Simbolo de la condición inicial en x
y0C = Symbol('y_0')                       #Símbolo de la condición inicial en y

In [15]:
ode1C = Eq(dxC,-aC*y1C(tC)*xC(tC))                     #Vamos a declarar las ecuaciones diferenciales para ver si se puede resolver con funciones y/o métodos de Python
ode1C

Eq(Derivative(x(t), t), -a*x(t)*y(t))

In [16]:
ode2C = Eq(dyC,-bC*xC(tC)*y1C(tC))                     #Vamos a declarar las ecuaciones diferenciales para ver si se puede resolver con funciones y/o métodos de Python
ode2C

Eq(Derivative(y(t), t), -b*x(t)*y(t))

In [17]:
dsolve(ode1C)  

Eq(x(t), C1*exp(-a*Integral(y(t), t)))

In [18]:
dsolve(ode2C)  

Eq(y(t), C1*exp(-b*Integral(x(t), t)))

In [42]:
#Definimos la función que caracteriza al sistema para poder utilizar otros métodos de resolución

def LanchesterC(estado, tiempo,a=a,b=b):
    l1 = -a*estado[1]*estado[0]                         
    l2 = -b*estado[0]*estado[1]
    return np.array([l1,l2])

In [20]:
# Usaremos el método de Runge-Kutta 2. Las variables son:
# y -> el estado inicial 
# t -> tiempo
# dt -> cambio en el tiempo
# derivadas -> la función que tiene el sistema

def RK2(y, t, dt, derivadas):
    k0 = dt*derivadas(y, t)
    k1 = dt*derivadas(y + k0, t + dt)
    y_next = y + 0.5*(k0 + k1)
    
    return y_next

In [21]:
N = 10 # -> definimos el número de steps

time = np.linspace(0,100,N)   # -> el tiempo

dt = 100/float(N-1)  # -> La variación del tiempo

yC = np.zeros([N,2], dtype=Symbol)   #Este arreglo va a tener las condiciones iniciales y va a guardar las soluciones
yC[0,0]=x0C
yC[0,1]=y0C

In [22]:
#Esto hace la resolución del método de Runge-Kutta2

for i in range(0,N-1):
    yC[i+1] = RK2(yC[i], time[i], dt, LanchesterC)

In [32]:
#Aquí vemos que el arreglo si tiene soluciones

yC[1,0]

-5.55555555555556*a*x_0*y_0 - 5.55555555555556*a*(-11.1111111111111*a*x_0*y_0 + x_0)*(-11.1111111111111*b*x_0*y_0 + y_0) + x_0

In [43]:
#Si x0=3y0
yC2 = np.zeros([N,2], dtype=Symbol) 
yC2[0,0]=3*y0C
yC2[0,1]=y0C

for i in range(0,N-1):
    yC2[i+1] = RK2(yC2[i], time[i], dt, LanchesterC)

In [44]:
yC2[1,0]

-16.6666666666667*a*y_0**2 - 5.55555555555556*a*(-33.3333333333333*a*y_0**2 + 3*y_0)*(-33.3333333333333*b*y_0**2 + y_0) + 3*y_0

- El modelo _VIETNAM_ es la unión de los últimos dos modelos: las tropas de **EU** contra el **Vietcong**
$$
\frac{dx}{dt} = -axy
$$
$$
\frac{dy}{dt} = -bx,
$$
donde $a$ es proporcional a la razón entre el área de un guerrillero $A_g \sim 2$ sq. ft. y el área ocupada por la guerrilla $A_x$
$$
a = c_1\frac{A_g}{A_x},
$$
un guerrillero cubre aproximadamente $1,000$ sq.ft. y si están dispersados $A_x = (1,000)sq.ft. \times x_0$.
$b$ representa la efectividad de la guerrilla contra una fuerza convencional, y depende de la probabilidad de que un disparo de un guerrillero mate a un soldado. 
$$
b = c_2p_x
$$
$c_1$ y $c_2$ son las tasas de disparo (depende de la tecnología del armamento) y se suponen aproximadas $c_1 \sim c_2$. ¿Cuáles son las condiciones de empate? ¿Cuántos soldados convencionales debe de tener el ejército convencional para derrotar a la guerrilla? En Vietnam, las tropas de EUA nunca fueron mayores a las del Vietcong por más de 6 a 1. ¿Pudo haber ganado EUA?


In [61]:
#VALORES PARA EL MODELO VIETNAM
tV = Symbol('t')                          #tiempo
xV = Function('x')                        #Número de tropas rojas 
dxV = Derivative(xV(tV), tV)                 #Cambio de la fuerza (número) de la tropa roja

y1V = Function('y')                        #Número de tropas azules
dyV = Derivative(y1V(tV), tV)                 #Cambio de la fuerza (número) de la tropa azul


aV = Symbol('a')                          #Potencia de fuego de x
bV = Symbol('b')                          #Potencia de fuego de y

Ag= Symbol('A_g')                         #Area de un guerrillero 
Ax= Symbol('A_x')                         #Area ocupada por la guerrilla 
c1,c2= symbols('c_1 c_2')                 #Tasas de disparo tal que c1~c2 (aproximadas)

px= Symbol('p_x')                         #Probabilidad de que un guerrillero mate a un soldado

x0V = Symbol('x_0')                       #Simbolo de la condición inicial en x
y0V = Symbol('y_0')                       #Símbolo de la condición inicial en y


In [64]:
#Condiciones: 
Ax=1000*x0
aV=c1*(Ag/Ax)
bV=c2*px

In [65]:
ode1V = Eq(dxV,-aV*y1V(tV)*xV(tV))                     #Vamos a declarar las ecuaciones diferenciales para ver si se puede resolver con funciones y/o métodos de Python
ode1V

Eq(Derivative(x(t), t), -A_g*c_1*x(t)*y(t)/(1000*x_0))

In [66]:
ode2V = Eq(dyV,-bV*xV(tV))                     #Vamos a declarar las ecuaciones diferenciales para ver si se puede resolver con funciones y/o métodos de Python
ode2V

Eq(Derivative(y(t), t), -c_2*p_x*x(t))

- Es posible modificar las ecuaciones para modelar combate convencional (_CONCON_):
$$
\frac{dx}{dt} = -cx-ay+P(t)
$$
$$
\frac{dy}{dt} = -bx-dy+Q(t)
$$
donde ${d,c}$ son la tasa de pérdidas operacionales (enfermedades, deserciones, etc.) -proporcional al número de las tropas, y ${a,b}$ es la tasa de pérdidas en combate. ${P,Q}$ es la tasa de refuerzos. 
La batalla de Iwo Jima, en la segunda guerra mundial, fué modelada por Engel en 1954, aplicando estas ecuaciones y dió una _comprobación empírica_ de las ecuaciones de Lanchester, aunque en este caso, sólo el ejército de EUA tuvo refuerzos:
$$
\frac{dx}{dt} = -ay
$$
$$
\frac{dy}{dt} = -bx+Q(t)
$$
Resuelva las ecuaciones con $x_0 = 21,500$, $y_0=0$ y 
$$Q(t) = 54,000 \mathcal{U}_{[0,1]} + 6,000 \mathcal{U}_{[2_3]} + 13,000 \mathcal{U}_{[5,6]},$$
donde $\mathcal{U}$ es la función escalón. 

- Encuentre los valores de $a$ y $b$ para ajustar los datos empíricos mostrados en la figura siguiente:

<img src="imagenes/iwo_jima.png"/>

- Si no hubiera habido refuerzos ¿Cuál hubiera sido el resultado de la batalla?
- ¿Puede sugerir (buscando alguna referencia de tropas cercanas japonesas) cómo intervenir para ganar esta batalla?

- **This is Sparta!** Es posible simular la batalla del Termópilas: Suponga que sólo $C$ unidades de cada lado caben en el estrecho (o paso) de Termópilas, entonces las ecuaciones se convierten en 
$$
\frac{dx}{dt} = -a \min(y,C)
$$
$$
\frac{dy}{dt} = -b \min(x,C)
$$
Separe en cuatro casos el espacio $x-y$ y dibuje las regiones de manera analítica. ¿Obtiene el mismo resultado numérico? Utilice los datos "históricos" ¿El resultado es parecido a la vida real?

- **Agentes** Use la clase agente para modelar el último escenario, suponga únicamente combate cuerpo a cuerpo, asigne una probabilidad de herir, morir y matar para los agentes que estén uno enfrente de otro. Agregue un valor de `cohesión` / `miedo`. Si pasa de un límite el miedo huye el agente. Agregue un atributo de `moral`. ¿Los resultados coinciden con el modelo de Lanchester?

## Preguntas extra
- Usando como base a **Chen, 2012** (Incluido en el repo) simule la batalla de Trafalgar con Lanchester.
- ¿Cómo modelaría fatiga o abastecimiento en Lanchester?
- ¿Cómo incluiría a un tercer combatiente? ¿Se ve afectadas las ecuaciones si los **rojos** tienen fuerzas regulares e irregulares?

## Bibliografía
- **Wikipedia** [_Lanchester laws_](http://en.wikipedia.org/wiki/Lanchester%27s_laws)
- **J.H. Engel** _A verification of Lanchester's Law_ `Journal of the Operations Research Society of America, Vol. 2, No. 2. (May, 1954), pp. 163-171`
- **Alex Chen** _This Means War! Modeling Combat with Applications to Real Time Strategy Games_ (2012)
- **Marcin Waniek** _An Agent-Based Simulation of the Battle of Kokenhausen_